# This code extract URL link for all the Laptops on the PCWorld.com

In [1]:
#Import BeautifulSoup and couple of other modules
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import numpy as np
import math

In [2]:
#Setting my_url to be the ALL Laptops Page on PCWorld.co.uk
shop_web_addr = 'https://www.pcworld.co.uk'
laptops_link = '/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/xx-criteria.html'
my_url = shop_web_addr+laptops_link
print(my_url)

https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/xx-criteria.html


In [3]:
#Extracting HTML from my_url (All Laptops) 
page = requests.get(my_url)
page_html = page.text
page_soup = soup(page_html, 'html.parser')

In [4]:
#Check the header of the the Page Soup
page_soup.h1

<h1 class="logo" id="logo">
<a href="https://www.pcworld.co.uk/gbuk/index.html" title="Back to home">
<img alt="Pcworld" src="https://pcw-ssl.cdn.dixons.com/grafx/gb/uk/header/logo/logo_pcw_logo.png"/>
</a>
</h1>

In [5]:
#Checking the number of the laptops listed at the time of inquiry
pagn_id = page_soup.find("span", {"class":"desktop"})
st=page_soup.find("strong")
st.contents[-1].split()[4]
print('There are in total', st.contents[-1].split()[4], 'Laptops Listed')

There are in total 317 Laptops Listed


In [6]:
data=page_soup.find("strong").contents[-1].split()
last_pagn = math.ceil(int(data[4])/int(data[2]))
print('Search has total', last_pagn, 'result pages')

Search has total 16 result pages


In [7]:
#Extracting Static Parts of the URL on each page
page_link_parts = page_soup.find("ul", {"class":"pagination"}).findAll('li')[1].a['href'].split('_20')
page_link_parts = [page_link_parts[0][:-1],'_20'+page_link_parts[1]]
print(page_link_parts)

['https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/', '_20/relevance-desc/xx-criteria.html']


In [8]:
#extracting a sample laptop link from the page
results = page_soup.findAll('article', {'class':'product result-prd'})
link=results[0].find('div', {'class':'desc'}).a['href']
link

'https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/hp-14-bp070sa-14-laptop-white-10169031-pdt.html'

In [9]:
#Extracting all Laptops URLs from all pages
urls = []
n=0
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
for pagn in list(range(1,last_pagn+1)):
    page_link = page_link_parts[0]+str(pagn)+page_link_parts[1]
    print(n, page_link)
    #Openning up connection, grabbing the page
    page = requests.get(page_link, headers=headers)
    while page.status_code == 503:
        #print('Service Unavailable "Response Code 503"')
        page = requests.get(page_link, headers=headers)
    print(page.status_code)
    page_html = page.text
    # Html Parsing
    page_soup = soup(page_html, 'html.parser')
    #print(page_soup)
    #print(page_soup.h1)
    page_soup.findAll('article', {'class':'product result-prd'})
    for result in results:
        try:
            urls.append(result.find('div', {'class':'desc'}).a['href'])
        except:
            print(result)
            pass
    n=n+1

0 https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/1_20/relevance-desc/xx-criteria.html
200
1 https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/2_20/relevance-desc/xx-criteria.html
200
2 https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/3_20/relevance-desc/xx-criteria.html
200
3 https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/4_20/relevance-desc/xx-criteria.html
200
4 https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/5_20/relevance-desc/xx-criteria.html
200
5 https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/6_20/relevance-desc/xx-criteria.html
200
6 https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/7_20/relevance-desc/xx-criteria.html
200
7 https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/703_7006_70006_xx_xx/8_20/relevance-desc/xx-criteria.html
200
8 https://www.pcworld.co.uk/gbuk

In [10]:
#Checking top 4 URLs
urls[:5]

['https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/hp-14-bp070sa-14-laptop-white-10169031-pdt.html',
 'https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/hp-14-bp069sa-14-laptop-black-10169033-pdt.html',
 'https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/hp-14-bp068sa-14-laptop-marine-blue-10169355-pdt.html',
 'https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/hp-14-bp059sa-14-laptop-jet-black-10166575-pdt.html',
 'https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/thomson-neo10-10-1-laptop-white-10175040-pdt.html']

In [11]:
#Pickle the URLs
import pickle

with open('PC_World_Urls.pickle', 'wb') as fp:
    pickle.dump(urls, fp)
